#Differential Evolution (DE)

#1. DE/rand/1

Standard differential evolution algorithm using a mutation strategy (rand/1) and a simple difference-based crossover.

In [ ]:
import numpy as np

# Define the objective function (example: Sphere function)
def objective_function(x):
    return np.sum(x**2)  # Sphere function: f(x) = sum(x_i^2)

def de_rand_1(objective_func, bounds, pop_size=50, max_iter=100, F=0.8, CR=0.9):
    """
    DE/rand/1 implementation for optimizing the objective_func.

    Parameters:
        objective_func (function): The objective function to be minimized.
        bounds (list of tuple): The search space bounds for each variable.
        pop_size (int): The population size (default: 50).
        max_iter (int): Maximum number of iterations (default: 100).
        F (float): The differential weight for mutation (default: 0.8).
        CR (float): The crossover probability (default: 0.9).

    Returns:
        tuple: (best_solution, best_fitness) where
            - best_solution (ndarray): The best solution found.
            - best_fitness (float): The fitness value of the best solution.
    """
    num_params = len(bounds)
    lower_bound, upper_bound = np.array(bounds).T

    # Initialize population
    population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_params))

    # Best solution found
    best_solution = None
    best_fitness = np.inf

    for i in range(max_iter):
        for j in range(pop_size):
            # Select three random indices different from current index j
            candidates = [idx for idx in range(pop_size) if idx != j]
            a, b, c = np.random.choice(candidates, 3, replace=False)

            # Mutation: DE/rand/1 mutation strategy
            mutant_vector = population[a] + F * (population[b] - population[c])

            # Clip the mutant vector to the bounds
            mutant_vector = np.clip(mutant_vector, lower_bound, upper_bound)

            # Crossover: Simple difference-based crossover
            trial_vector = np.where(np.random.rand(num_params) <= CR, mutant_vector, population[j])

            # Evaluate the trial vector
            trial_fitness = objective_func(trial_vector)

            # Update the population
            if trial_fitness < objective_func(population[j]):
                population[j] = trial_vector

                # Update the best solution found
                if trial_fitness < best_fitness:
                    best_solution = trial_vector
                    best_fitness = trial_fitness

    return best_solution, best_fitness

# Example usage:
np.random.seed(42)
bounds = [(-5.12, 5.12)] * 5  # Bounds for each parameter
best_solution, best_fitness = de_rand_1(objective_function, bounds)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Best Solution: [-2.81684237e-03 -1.87596541e-03 -7.87042141e-05  1.11259623e-03
  3.42745988e-03]
Best Fitness: 2.4445393158404352e-05


#2. DE/best/1

Uses the best individual in the current population as a donor for mutation.

In [ ]:
import numpy as np

def de_best_1(objective_func, bounds, pop_size=50, max_iter=100, F=0.8, CR=0.9):
    """
    DE/best/1 implementation for optimizing the objective_func.

    Parameters:
        objective_func (function): The objective function to be minimized.
        bounds (list of tuple): The search space bounds for each variable.
        pop_size (int): The population size (default: 50).
        max_iter (int): Maximum number of iterations (default: 100).
        F (float): The differential weight for mutation (default: 0.8).
        CR (float): The crossover probability (default: 0.9).

    Returns:
        tuple: (best_solution, best_fitness) where
            - best_solution (ndarray): The best solution found.
            - best_fitness (float): The fitness value of the best solution.
    """
    num_params = len(bounds)
    lower_bound, upper_bound = np.array(bounds).T

    # Initialize population
    population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_params))

    # Best solution found
    best_solution = None
    best_fitness = np.inf

    for i in range(max_iter):
        # Find the index of the best individual in the population
        best_index = np.argmin([objective_func(individual) for individual in population])
        best_individual = population[best_index]

        for j in range(pop_size):
            # Mutation: DE/best/1 mutation strategy
            mutant_vector = best_individual + F * (population[np.random.choice(pop_size)] - population[np.random.choice(pop_size)])

            # Clip the mutant vector to the bounds
            mutant_vector = np.clip(mutant_vector, lower_bound, upper_bound)

            # Crossover: Simple difference-based crossover
            trial_vector = np.where(np.random.rand(num_params) <= CR, mutant_vector, population[j])

            # Evaluate the trial vector
            trial_fitness = objective_func(trial_vector)

            # Update the population
            if trial_fitness < objective_func(population[j]):
                population[j] = trial_vector

                # Update the best solution found
                if trial_fitness < best_fitness:
                    best_solution = trial_vector
                    best_fitness = trial_fitness

    return best_solution, best_fitness

# Example usage:
np.random.seed(42)
bounds = [(-5.12, 5.12)] * 5  # Bounds for each parameter
best_solution, best_fitness = de_best_1(objective_function, bounds)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Best Solution: [-2.52265066e-08 -2.12027597e-08 -3.10403810e-08  7.26964720e-08
  3.80006149e-08]
Best Fitness: 8.778262682446304e-15


#3. DE/current-to-best/1

Combines the best individual and the current individual to guide mutation.

In [ ]:
import numpy as np

def de_current_to_best_1(objective_func, bounds, pop_size=50, max_iter=100, F=0.8, CR=0.9):
    """
    DE/current-to-best/1 implementation for optimizing the objective_func.

    Parameters:
        objective_func (function): The objective function to be minimized.
        bounds (list of tuple): The search space bounds for each variable.
        pop_size (int): The population size (default: 50).
        max_iter (int): Maximum number of iterations (default: 100).
        F (float): The differential weight for mutation (default: 0.8).
        CR (float): The crossover probability (default: 0.9).

    Returns:
        tuple: (best_solution, best_fitness) where
            - best_solution (ndarray): The best solution found.
            - best_fitness (float): The fitness value of the best solution.
    """
    num_params = len(bounds)
    lower_bound, upper_bound = np.array(bounds).T

    # Initialize population
    population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_params))

    # Best solution found
    best_solution = None
    best_fitness = np.inf

    for i in range(max_iter):
        # Find the index of the best individual in the population
        best_index = np.argmin([objective_func(individual) for individual in population])
        best_individual = population[best_index]

        for j in range(pop_size):
            # Mutation: DE/current-to-best/1 mutation strategy
            mutant_vector = population[j] + F * (best_individual - population[j]) + F * (population[np.random.choice(pop_size)] - population[np.random.choice(pop_size)])

            # Clip the mutant vector to the bounds
            mutant_vector = np.clip(mutant_vector, lower_bound, upper_bound)

            # Crossover: Simple difference-based crossover
            trial_vector = np.where(np.random.rand(num_params) <= CR, mutant_vector, population[j])

            # Evaluate the trial vector
            trial_fitness = objective_func(trial_vector)

            # Update the population
            if trial_fitness < objective_func(population[j]):
                population[j] = trial_vector

                # Update the best solution found
                if trial_fitness < best_fitness:
                    best_solution = trial_vector
                    best_fitness = trial_fitness

    return best_solution, best_fitness

# Example usage:
np.random.seed(42)
bounds = [(-5.12, 5.12)] * 5  # Bounds for each parameter
best_solution, best_fitness = de_current_to_best_1(objective_function, bounds)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Best Solution: [-9.81080694e-08 -4.47491170e-08  1.01961509e-08 -1.70464370e-08
 -3.29536824e-10]
Best Fitness: 1.2022327857396713e-14


#4. DE/rand-to-best/2

Merges random individuals with the best individual to generate mutant vectors.

In [ ]:
import numpy as np

def de_rand_to_best_2(objective_func, bounds, pop_size=50, max_iter=100, F=0.8, CR=0.9):
    """
    DE/rand-to-best/2 implementation for optimizing the objective_func.

    Parameters:
        objective_func (function): The objective function to be minimized.
        bounds (list of tuple): The search space bounds for each variable.
        pop_size (int): The population size (default: 50).
        max_iter (int): Maximum number of iterations (default: 100).
        F (float): The differential weight for mutation (default: 0.8).
        CR (float): The crossover probability (default: 0.9).

    Returns:
        tuple: (best_solution, best_fitness) where
            - best_solution (ndarray): The best solution found.
            - best_fitness (float): The fitness value of the best solution.
    """
    num_params = len(bounds)
    lower_bound, upper_bound = np.array(bounds).T

    # Initialize population
    population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_params))

    # Best solution found
    best_solution = None
    best_fitness = np.inf

    for i in range(max_iter):
        # Find the index of the best individual in the population
        best_index = np.argmin([objective_func(individual) for individual in population])
        best_individual = population[best_index]

        for j in range(pop_size):
            # Mutation: DE/rand-to-best/2 mutation strategy
            a, b, c = np.random.choice(pop_size, 3, replace=False)

            mutant_vector = population[a] + F * (population[b] - population[c]) + F * (best_individual - population[j])

            # Clip the mutant vector to the bounds
            mutant_vector = np.clip(mutant_vector, lower_bound, upper_bound)

            # Crossover: Simple difference-based crossover
            trial_vector = np.where(np.random.rand(num_params) <= CR, mutant_vector, population[j])

            # Evaluate the trial vector
            trial_fitness = objective_func(trial_vector)

            # Update the population
            if trial_fitness < objective_func(population[j]):
                population[j] = trial_vector

                # Update the best solution found
                if trial_fitness < best_fitness:
                    best_solution = trial_vector
                    best_fitness = trial_fitness

    return best_solution, best_fitness

# Example usage:
np.random.seed(42)
bounds = [(-5.12, 5.12)] * 5  # Bounds for each parameter
best_solution, best_fitness = de_rand_to_best_2(objective_function, bounds)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)


Best Solution: [-0.03665746  0.10916247  0.0517539  -0.0641463  -0.04215531]
Best Fitness: 0.02183049815481654


# 5. Self-adaptive Differential Evolution:
DE variants that adaptively adjust control parameters during evolution.

In [ ]:
import numpy as np

def sade(objective_func, bounds, pop_size=50, max_iter=100):
    """
    Self-adaptive Differential Evolution (SaDE) implementation for optimizing the objective_func.

    Parameters:
        objective_func (function): The objective function to be minimized.
        bounds (list of tuple): The search space bounds for each variable.
        pop_size (int): The population size (default: 50).
        max_iter (int): Maximum number of iterations (default: 100).

    Returns:
        tuple: (best_solution, best_fitness) where
            - best_solution (ndarray): The best solution found.
            - best_fitness (float): The fitness value of the best solution.
    """
    num_params = len(bounds)
    lower_bound, upper_bound = np.array(bounds).T

    # Initialize population
    population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_params))

    # Initialize mutation scale factor (F) and crossover probability (CR) arrays
    F = np.random.uniform(0.5, 1.0, size=pop_size)
    CR = np.random.uniform(0.0, 1.0, size=pop_size)

    # Best solution found
    best_solution = None
    best_fitness = np.inf

    for i in range(max_iter):
        for j in range(pop_size):
            # Select three distinct random indices for mutation
            candidates = [idx for idx in range(pop_size) if idx != j]
            a, b, c = np.random.choice(candidates, 3, replace=False)

            # Generate mutant vector using adaptive F
            mutant_vector = population[a] + F[j] * (population[b] - population[c])

            # Clip mutant vector to bounds
            mutant_vector = np.clip(mutant_vector, lower_bound, upper_bound)

            # Perform crossover to create trial vector using adaptive CR
            trial_vector = np.where(np.random.rand(num_params) <= CR[j], mutant_vector, population[j])

            # Evaluate the trial vector
            trial_fitness = objective_func(trial_vector)

            # Update the population based on trial solution
            if trial_fitness < objective_func(population[j]):
                population[j] = trial_vector
                F[j] = F[j] + 0.1  # Increase F if trial is successful
                CR[j] = CR[j] + 0.1  # Increase CR if trial is successful

                # Update the best solution found
                if trial_fitness < best_fitness:
                    best_solution = trial_vector
                    best_fitness = trial_fitness
            else:
                F[j] = F[j] - 0.1  # Decrease F if trial is unsuccessful
                CR[j] = CR[j] - 0.1  # Decrease CR if trial is unsuccessful

                F[j] = np.clip(F[j], 0.5, 1.0)  # Clip F to [0.5, 1.0]
                CR[j] = np.clip(CR[j], 0.0, 1.0)  # Clip CR to [0.0, 1.0]

    return best_solution, best_fitness

# Example usage:
np.random.seed(42)
bounds = [(-5.12, 5.12)] * 5  # Bounds for each parameter
best_solution, best_fitness = sade(objective_function, bounds)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)

Best Solution: [-0.28741423 -0.13493397  0.65902697 -1.406934    0.13066659]
Best Fitness: 2.531667692515466
